In [13]:
from src import MACGenerator, UDP_TX, SlidingBook
import src.TX as TX

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import socket
import struct
import time
import json
import cv2

import sys

sys.path.append('..')
import utils.utils as utils
import utils.Auth as Auth








def tx_param(IP, PORT, param:dict):
# send the parameters to the receiver and the time.time() to synchronize the sender and the receiver
    time.sleep(.1)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        while True:
            try:
                s.connect((IP, PORT))
                break
            except:
                pass
            
        data = json.dumps(param).encode()
        for i in range(len(data)//1000):
            s.send(data[i*1000:(i+1)*1000])
            time.sleep(.01)
        s.send(data[(i+1)*1000:])
        time.sleep(.1)
        s.send(b'END')
        s.close()
    
    time.sleep(.1)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        while True:
            try:
                s.connect((IP, PORT))
                break
            except:
                pass
        data = time.time()
        data = struct.pack('d', data)
        s.send(data)
        s.close()

    time.sleep(.1)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        while True:
            try:
                s.connect((IP, PORT))
                break
            except:
                pass
        
        res = s.recv(1024)
        if res == b'ABORT':
            print("The receiver has aborted the connection")
        elif res == b'RETRY':
            print("The receiver has requested to resend the parameters")
        s.close()
    
    if res == b'ABORT':
        return False
    elif res == b'RETRY':
        time.sleep(2)
        print("retrying to send the parameters!60")
        return tx_param(IP, PORT, param)
    elif res == b'OK':
        return True
    else:
        print("The receiver has sent an unknown message")
        return False



def run_experiment(IP, PORT, param:dict):
    # Capture video from the default camera
    cap = cv2.VideoCapture(0) 
    # Define the compression parameters for Progressive JPEG
    compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, param['QUALITY']]
    ## counting the number of frames per second

    start_time = time.time()
    runTime_cnt = time.time()
    frame_counter = 0
    average_tx_size = 0
    while True:
        ret, frame = cap.read()
        # Encode the image to JPEG format in memory
        success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
        data = encoded_frame.tobytes()
        average_tx_size+=len(data)//1000

        #############################  TX  ########################################
        buffer = SlidingBook(num_pages=40, page_size=len(param['X']))
        page_processor = MACGenerator(X = param['X'], Y = param['Y'], secret_key=param['KEY'].encode(), digestmod=param['DIGESTMOD'])
        udp_tx = UDP_TX(IP= IP, PORT= PORT, buffer=buffer, page_processor= page_processor, Payload_Size_Byte = param['PAYLOAD_SIZE_BYTE'])
        pages = udp_tx.segment_to_pages(data)
        udp_tx.transmit(pages, param['ATTACK_PROBABILITY'], delay=param['DELAY'])
        ###########################################################################

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
            break

        # print the frame rate
        frame_counter += 1
        if time.time() - start_time >= 1:
            print("frame per second: ", frame_counter, "fps ,data rate: ", average_tx_size, "KB/s")
            frame_counter = 0
            start_time = time.time()
            average_tx_size = 0
        if time.time() - runTime_cnt >= param['DURATION']:
            break
    cv2.destroyAllWindows()
    cap.release()
    



#### test the function
###### IP, PORRT ######
IP = "10.42.0.15"
PORT = 23422
#######################

### parameters that needs to be exhanged between the sender and the receiver #####

# X = np.array([  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
#                 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
#                 ])


# X = Auth.X

X = np.eye(20)

Y = Auth.Get_Y(X)
for p in [.01, .05, .1, .15, .2, .25, .3]:
    param = {}
    param['X'] = X.tolist()
    param['Y'] = Y.tolist()
    param['KEY'] = "key"
    param['DIGESTMOD'] = 'sha256'
    param['PAYLOAD_SIZE_BYTE']= 16
    param['QUALITY'] = 10   # param['QUALITY'] of the JPEG compression
    param['ATTACK_PROBABILITY'] = p  # probability of the attack
    param['DURATION'] = 30  # running time of the sender in seconds
    ########## added delay for stability reasons in 5G to reduce the fps ################
    param['DELAY'] = 0.01
    ####################################################################################
    res = tx_param(IP, PORT, param)
    if res:
        run_experiment(IP, PORT, param)

[ WARN:0@1628.694] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  14 fps ,data rate:  112 KB/s
frame per second:  24 fps ,data rate:  192 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  25 fps ,data rate:  200 KB/s
frame per second:  26 fps ,data rate:  2

[ WARN:0@1659.516] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  19 fps ,data rate:  152 KB/s
frame per second:  23 fps ,data rate:  184 KB/s
frame per second:  21 fps ,data rate:  168 KB/s
frame per second:  24 fps ,data rate:  192 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  2

[ WARN:0@1690.362] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


...

frame per second:  25 fps ,data rate:  200 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  25 fps ,data rate:  200 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  25 fps ,data rate:  200 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  29 fps ,data rate:  2

[ WARN:0@1782.903] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  23 fps ,data rate:  181 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  26 fps ,data rate:  208 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  25 fps ,data rate:  200 KB/s
frame per second:  21 fps ,data rate:  168 KB/s
frame per second:  15 fps ,data rate:  120 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  31 fps ,data rate:  248 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  30 fps ,data rate:  240 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  30 fps ,data rate:  2

[ WARN:0@1813.856] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  24 fps ,data rate:  191 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  27 fps ,data rate:  216 KB/s
frame per second:  23 fps ,data rate:  184 KB/s
frame per second:  23 fps ,data rate:  184 KB/s
frame per second:  20 fps ,data rate:  160 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  29 fps ,data rate:  232 KB/s
frame per second:  28 fps ,data rate:  224 KB/s
frame per second:  28 fps ,data rate:  2

In [2]:
###### IP, PORRT ######
IP = "10.42.0.15"
PORT = 23422
#######################



################# number of experimnets ###############
qualities = [25, 75]
payload_sizes = [20, 40]
attack_probabilities = [0, 0.01, 0.1]

# qualities = [60]
# payload_sizes = [32, 128]
# attack_probabilities = [0, 0.001]

digets = ['sha384', 'sha512']
Duration = 30 # Seconds
exp = utils.Load_Experiments("../Examples/Xs.pkl")
#######################################################

total_number_of_experiments = 0
for i in range(len(exp)):
    if not(exp[i]['parameters']['m_nr'] == 20 and exp[i]['parameters']['t_nr'] != 20 and exp[i]['parameters']['t_nr'] == 10):
        continue
    if np.array_equal(np.array(exp[i]['results']['X']),np.ones(np.array(exp[i]['results']['X']).shape)) or exp[i]['parameters']['q'] != 1:
        continue
    total_number_of_experiments += 1

total_number_of_experiments = len(qualities)*len(payload_sizes)*len(attack_probabilities)*len(digets)*total_number_of_experiments
print("Total runtime for experiments: ", total_number_of_experiments*Duration/60, "minutes")

cnt = 0
for i in range(len(exp)):
    if not(exp[i]['parameters']['m_nr'] == 20 and exp[i]['parameters']['t_nr'] != 20 and exp[i]['parameters']['t_nr'] == 10):
        continue
    if np.array_equal(np.array(exp[i]['results']['X']),np.ones(np.array(exp[i]['results']['X']).shape)) or exp[i]['parameters']['q'] != 1:
        continue

    X = exp[i]['results']['X']
    Y = utils.Get_Y(X)
    # try:
    #     if Y := utils.Get_Y(X) is None:
    #         continue
    # except:
    #     continue

    for quality in qualities:
        for payload_size in payload_sizes:
            for attack_prob in attack_probabilities:
                for digest in digets:
                    print(f"x:{X}")
                    param = {}
                    param['X'] = X.tolist()
                    try:
                        param['Y'] = Y.tolist()
                    except:
                        continue
                    # param['Y'] = Y.tolist()
                    param['KEY'] = "key"
                    param['DIGESTMOD'] = digest
                    param['PAYLOAD_SIZE_BYTE'] = payload_size
                    param['QUALITY'] = quality
                    param['ATTACK_PROBABILITY'] = attack_prob
                    param['DURATION'] = Duration

                    # plt.figure(figsize = (10,5), dpi=80)
                    # plt.subplot(1, 2, 1)
                    # sns.heatmap(param['X'])
                    # plt.title('X')
                    # plt.subplot(1, 2, 2)
                    # try:
                    #     sns.heatmap(param['Y'])
                    #     plt.title('Y')
                    # except:
                    #     pass
                    # cnt += 1
                    # plt.suptitle(f'Running Experiment {cnt} out of {total_number_of_experiments}' + ' - Quality:' + str(quality) + ' - Payload:' + str(payload_size) + ' - Prob:' + str(attack_prob) + ' - Digest:' + digest)
                    # plt.show()

                    res = tx_param(IP, PORT, param)
                    time.sleep(.1)
                    if res:
                        run_experiment(IP, PORT, param)
                    time.sleep(.1)


                    

           




Total runtime for experiments:  48.0 minutes
x:[[1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 0. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 1. 1. 1.]
 [1. 1. 1. 0. 0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 1. 1. 0. 1.]
 [1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 0. 1. 0. 1.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 1. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]]
The receiver has aborted the connection
x:[[1. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 1. 1. 1. 0. 1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1. 0. 1. 1. 0. 0.]
 [0. 0. 1. 1. 0. 1. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1.

[ WARN:0@3.234] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  25 fps ,data rate:  449 KB/s
frame per second:  30 fps ,data rate:  512 KB/s
frame per second:  31 fps ,data rate:  527 KB/s
frame per second:  29 fps ,data rate:  493 KB/s
frame per second:  30 fps ,data rate:  510 KB/s
frame per second:  30 fps ,data rate:  510 KB/s
frame per second:  31 fps ,data rate:  527 KB/s
frame per second:  29 fps ,data rate:  493 KB/s
frame per second:  31 fps ,data rate:  528 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  31 fps ,data rate:  558 KB/s
frame per second:  31 fps ,data rate:  558 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  28 fps ,data rate:  504 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  30 fps ,data rate:  536 KB/s
frame per second:  31 fps ,data rate:  553 KB/s
frame per second:  31 fps ,data rate:  511 KB/s
frame per second:  31 fps ,data rate:  496 KB/s
frame per second:  31 fps ,data rate:  5

[ WARN:0@34.326] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  28 fps ,data rate:  488 KB/s
frame per second:  30 fps ,data rate:  537 KB/s
frame per second:  31 fps ,data rate:  555 KB/s
frame per second:  31 fps ,data rate:  557 KB/s
frame per second:  30 fps ,data rate:  528 KB/s
frame per second:  30 fps ,data rate:  518 KB/s
frame per second:  30 fps ,data rate:  556 KB/s
frame per second:  31 fps ,data rate:  556 KB/s
frame per second:  30 fps ,data rate:  534 KB/s
frame per second:  31 fps ,data rate:  517 KB/s
frame per second:  30 fps ,data rate:  508 KB/s
frame per second:  30 fps ,data rate:  519 KB/s
frame per second:  29 fps ,data rate:  500 KB/s
frame per second:  29 fps ,data rate:  522 KB/s
frame per second:  31 fps ,data rate:  558 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  31 fps ,data rate:  558 KB/s
frame per second:  30 fps ,data rate:  540 KB/s
frame per second:  30 fps ,data rate:  5

...

frame per second:  24 fps ,data rate:  999 KB/s
frame per second:  24 fps ,data rate:  987 KB/s
frame per second:  26 fps ,data rate:  1067 KB/s
frame per second:  26 fps ,data rate:  1067 KB/s
frame per second:  29 fps ,data rate:  1140 KB/s
frame per second:  26 fps ,data rate:  1053 KB/s
frame per second:  26 fps ,data rate:  1091 KB/s
frame per second:  26 fps ,data rate:  1092 KB/s
frame per second:  26 fps ,data rate:  1092 KB/s
frame per second:  28 fps ,data rate:  1180 KB/s
frame per second:  27 fps ,data rate:  1134 KB/s
frame per second:  27 fps ,data rate:  1117 KB/s
frame per second:  25 fps ,data rate:  1025 KB/s
frame per second:  26 fps ,data rate:  1065 KB/s
frame per second:  29 fps ,data rate:  1199 KB/s
frame per second:  30 fps ,data rate:  1253 KB/s
frame per second:  28 fps ,data rate:  1173 KB/s
frame per second:  24 fps ,data rate:  1002 KB/s
frame per second:  24 fps ,data rate:  1002 KB/s
frame per second:  27 fps ,data rate:  1116 KB/s
frame per second:  29 

[ WARN:0@2866.610] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  30 fps ,data rate:  1219 KB/s
frame per second:  30 fps ,data rate:  1238 KB/s
frame per second:  30 fps ,data rate:  1223 KB/s
frame per second:  28 fps ,data rate:  1159 KB/s
frame per second:  30 fps ,data rate:  1245 KB/s
frame per second:  29 fps ,data rate:  1191 KB/s
frame per second:  24 fps ,data rate:  993 KB/s
frame per second:  27 fps ,data rate:  1123 KB/s
frame per second:  24 fps ,data rate:  999 KB/s
frame per second:  27 fps ,data rate:  1104 KB/s
frame per second:  28 fps ,data rate:  1161 KB/s
frame per second:  28 fps ,data rate:  1157 KB/s
frame per second:  27 fps ,data rate:  1122 KB/s
frame per second:  29 fps ,data rate:  1218 KB/s
frame per second:  29 fps ,data rate:  1209 KB/s
frame per second:  27 fps ,data rate:  1124 KB/s
frame per second:  27 fps ,data rate:  1160 KB/s
frame per second:  28 fps ,data rate:  1204 KB/s
frame per second:  28 fps ,data rate:  1204 KB/s
frame per second:  29 fps ,data rate:  1247 KB/s
frame per second:  28 

[ WARN:0@2897.694] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  28 fps ,data rate:  1174 KB/s
frame per second:  28 fps ,data rate:  1176 KB/s
frame per second:  27 fps ,data rate:  1134 KB/s
frame per second:  25 fps ,data rate:  1050 KB/s
frame per second:  29 fps ,data rate:  1218 KB/s
frame per second:  29 fps ,data rate:  1218 KB/s
frame per second:  29 fps ,data rate:  1216 KB/s
frame per second:  29 fps ,data rate:  1218 KB/s
frame per second:  24 fps ,data rate:  988 KB/s
frame per second:  27 fps ,data rate:  1122 KB/s
frame per second:  27 fps ,data rate:  1161 KB/s
frame per second:  28 fps ,data rate:  1204 KB/s
frame per second:  26 fps ,data rate:  1118 KB/s
frame per second:  30 fps ,data rate:  1290 KB/s
frame per second:  29 fps ,data rate:  1247 KB/s
frame per second:  26 fps ,data rate:  1118 KB/s
frame per second:  26 fps ,data rate:  1118 KB/s
frame per second:  24 fps ,data rate:  1032 KB/s
frame per second:  28 fps ,data rate:  1204 KB/s
frame per second:  29 fps ,data rate:  1247 KB/s
frame per second:  27